### Load base and KSI daily

In [10]:
from pathlib import Path
import pandas as pd

REPO_ROOT = Path.cwd().parents[0] if Path.cwd().name == "notebooks" else Path.cwd()
DATA_DIR = REPO_ROOT / "data"
INTERIM_DIR = DATA_DIR / "interim"
PROCESSED_DIR = DATA_DIR / "processed"

base = pd.read_parquet(PROCESSED_DIR / "base_nbhd_day_w158.parquet")
ksi = pd.read_parquet(INTERIM_DIR / "ksi_nbhd_daily.parquet")

print("base:", base.shape)
print("ksi:", ksi.shape)

base.head(2)


base: (692514, 7)
ksi: (4879, 8)


,AREA_ID,AREA_NAME,date,collision_count,area_id,nbhd_id,area_name
0,2502366,South Eglinton-Davisville,2014-01-01,0,2502366,174,South Eglinton-Davisville
1,2502366,South Eglinton-Davisville,2014-01-02,0,2502366,174,South Eglinton-Davisville


### Merge, fill zeros and save

In [11]:
# To Ensure date columns are identical types - datetime objects
base["date"] = pd.to_datetime(base["date"])
ksi["date"] = pd.to_datetime(ksi["date"])

#Ensure nbhd_id columns are identical types
base["nbhd_id"] = pd.to_numeric(base["nbhd_id"], errors="coerce").astype(int)
ksi["nbhd_id"] = pd.to_numeric(ksi["nbhd_id"], errors="coerce").astype(int)

#renaming collision_count to collisions in base to match print statement later
#previous output showed collision_count but print statement asked for collisions
if "collision_count" in base.columns:
    base = base.rename(columns={"collision_count": "collisions"})

# Merge
merged = base.merge(
    ksi,
    on=["date", "nbhd_id"],
    how="left"
)

#Fill NaNs with 0 for days where no KSI collisions occurred
ksi_cols = [
    "ksi_collisions", "ksi_fatal_collisions", "ksi_serious_collisions",
    "ksi_fatal_victims", "ksi_victim_count", "ksi_weighted_score"
]
for c in ksi_cols:
    merged[c] = merged[c].fillna(0).astype(int) #patch: was previously as float. added astype to convert to int

# Output save
out_path = PROCESSED_DIR / "gold_nbhd_day_partial.parquet"
merged.to_parquet(out_path, index=False)
print("Saved:", out_path)

# Verifying the output
merged[["date", "nbhd_id", "collisions"] + ksi_cols].head(10)

Saved: C:\code\pyspark-playground\Covercheck-Toronto\data\processed\gold_nbhd_day_partial.parquet


,date,nbhd_id,collisions,ksi_collisions,ksi_fatal_collisions,ksi_serious_collisions,ksi_fatal_victims,ksi_victim_count,ksi_weighted_score
0,2014-01-01,174,0,0,0,0,0,0,0
1,2014-01-02,174,0,0,0,0,0,0,0
2,2014-01-03,174,2,0,0,0,0,0,0
3,2014-01-04,174,0,0,0,0,0,0,0
4,2014-01-05,174,1,0,0,0,0,0,0
5,2014-01-06,174,1,0,0,0,0,0,0
6,2014-01-07,174,0,0,0,0,0,0,0
7,2014-01-08,174,0,0,0,0,0,0,0
8,2014-01-09,174,2,0,0,0,0,0,0
9,2014-01-10,174,1,0,0,0,0,0,0


In [12]:
merged[["date", "nbhd_id", "collisions"] + ksi_cols].describe()

,date,nbhd_id,collisions,ksi_collisions,ksi_fatal_collisions,ksi_serious_collisions,ksi_fatal_victims,ksi_victim_count,ksi_weighted_score
count,692514,692514.000000,692514.000000,692514.000000,692514.000000,692514.000000,692514.000000,692514.000000,692514.000000
mean,2020-01-01 00:00:00,88.107595,0.952180,0.002137,0.000349,0.001821,0.000355,0.005245,0.004690
min,2014-01-01 00:00:00,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2016-12-31 00:00:00,43.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,2020-01-01 00:00:00,88.500000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,2023-01-01 00:00:00,134.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,2025-12-31 00:00:00,174.000000,28.000000,2.000000,2.000000,2.000000,3.000000,18.000000,6.000000
std,NaN,51.074020,1.274196,0.046398,0.018767,0.042768,0.019298,0.128736,0.104461


In [13]:
merged.shape

(692514, 13)